In [50]:
from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.prompts import PromptTemplate
from pathlib import Path

import os
from dotenv import load_dotenv

load_dotenv()  # carga desde .env

# opcionalmente, puedes verificar:
assert os.getenv("OPENAI_API_KEY"), "No se encontró OPENAI_API_KEY"
# 1. Cargar base de datos
db_path = Path("C:/Users/05457936J/Desktop/CORAL/opositIA/data/outputs/oposiciones.db").resolve()
sql_database = SQLDatabase.from_uri(f"sqlite:///{db_path.as_posix()}")

# 2. Crear cliente OpenAI (usando GPT-4.1-nano, por ejemplo)
llm = OpenAI(model="gpt-4.1-nano", temperature=0.1,
             api_key=os.getenv("OPENAI_API_KEY"))

from llama_index.core.prompts import PromptTemplate

prompt_template = PromptTemplate(
    """You are an expert SQL assistant for a **SQLite database que contiene información oficial sobre oposiciones públicas**.

The database has one main table called `oposiciones` with exactly these columns (use them verbatim, case-sensitive):

- titulo
- organismo_que_convoca
- titulacion_requerida
- via
- plazas_convocadas
- plazas_libres
- otras_plazas
- fecha_de_publicacion
- fecha_de_cierre
- referencia

Your task is to transform a question written in natural Spanish into a syntactically correct SQLite query that retrieves data relevant to the question, and return the answer.

---

### Important rules

1. **Plazas vigentes (obligatorio)**
   Every query must include a filter to return only current/open positions:
   fecha_de_cierre >= DATE('now')
2. **Fuzzy matching**
If the user uses synonyms or general terms (e.g. "investigador"), match loosely:
LOWER(titulo) LIKE '%investigador%'
OR LOWER(titulo) LIKE '%investigacion%'
3. **Retrieve all results (no LIMIT)**
Return the full result set. Do not use LIMIT unless the user explicitly asks for it.

4. **No SELECT ***
Only query the specific columns needed to answer the question.

5. **Output format (una línea por sección)**
Question: <pregunta original>
SQLQuery: <consulta SQL generada>
SQLResult: <resultado de la consulta>
Answer: <respuesta final explicativa>
6. **Use only columns listed**. Qualify names if needed. Dialect is SQLite (fechas en formato 'YYYY-MM-DD').
7. **Return Referenica** Always include the referencia column in the SELECT clause, regardless of the user's question. This field uniquely identifies each row and must always be returned in the results.

Use only the following schema:

{schema}

---

Question: {query_str}
SQLQuery:"""
)
